In [3]:
import pandas as pd
import numpy as np
import pickle
import datetime
from tqdm import tqdm

In [5]:
# import data
predicted_df = pd.read_pickle("expected_returns.pkl")

file = open("data_prices.pkl", 'rb')
raw_prices = pickle.load(file)

In [11]:
# create dataframe for all predicted prices

predicted_dfs = list()
actual_dfs = list()

for i in tqdm(list(predicted_df.keys())):
    predicted_temp = pd.DataFrame.from_dict(predicted_df[i]).loc['predicted_finetuned',:]
    actual_temp    = pd.DataFrame.from_dict(predicted_df[i]).loc['actual',:]
    predicted_temp = pd.DataFrame.from_dict(dict(zip(predicted_temp.index, predicted_temp.values)))
    actual_temp = pd.DataFrame.from_dict(dict(zip(actual_temp.index, actual_temp.values)))
    
    predicted_dfs.append(predicted_temp)
    actual_dfs.append(actual_temp)
    
    
predictprc_df = pd.concat(predicted_dfs)
#actual_df = pd.concat(actual_dfs)
predictprc_df = predictprc_df.set_index(raw_prices[raw_prices.index>'2019-12-30'].index)
#actual_df     = actual_df.    set_index(raw_prices[raw_prices.index>'2019-12-30'].index)    
    
    

100%|███████████████████████████████████████████| 34/34 [00:00<00:00, 43.33it/s]


In [14]:
raw_price_df = raw_prices[raw_prices.index>'2019-12-29'].iloc[:, raw_prices.columns.get_level_values(0)=='Adj Close']

raw_price_df.columns = raw_price_df.columns.droplevel(level=0)

raw_price_df = raw_price_df[raw_price_df.columns.intersection(set(predictprc_df.columns))]

In [15]:
predictprc_df = predictprc_df.add_suffix('_predict')
raw_price_df  = raw_price_df .add_suffix('_actual')

In [24]:
predictprc_df['predicting_day'] = 0
predictprc_df.iloc[20::21, [-1]] = 1

In [29]:
raw_price_df = raw_price_df.merge(predictprc_df, left_index=True, right_index=True, how = 'left')

In [33]:
raw_price_df.iloc[0, [-1]] = 1

In [109]:
hit_rate_df = raw_price_df[raw_price_df.predicting_day==1]


In [110]:
# calculate hit rate (check whether (predicted_t+1 - actual_t) and (actual_t+1 - actual_t) are in the same direction)

trading_threshold = 0.0005
ticker_list = list(predictprc_df.columns[:-1].str.rstrip('_predict'))

for ticker_i in ticker_list:
    hit_rate_df[ticker_i+'_actualret']  = hit_rate_df[ticker_i+'_actual'].diff().shift(-1)/\
    hit_rate_df[ticker_i+'_actual']
    
    hit_rate_df[ticker_i+'_predictret'] = (hit_rate_df[ticker_i+'_predict'].shift(-1) - \
                                            hit_rate_df[ticker_i+'_actual']) / hit_rate_df[ticker_i+'_actual']
    
    hit_rate_df[ticker_i+'_actualsignal'] = 0
    hit_rate_df[ticker_i+'_actualsignal'] = np.where(hit_rate_df[
        ticker_i+'_actualret']>=trading_threshold, 1,hit_rate_df[ticker_i+'_actualsignal'] )
    hit_rate_df[ticker_i+'_actualsignal'] = np.where(hit_rate_df[
        ticker_i+'_actualret']<=(-1)*trading_threshold, -1,hit_rate_df[ticker_i+'_actualsignal'] )
    
    
    hit_rate_df[ticker_i+'_predictsignal'] = 0
    hit_rate_df[ticker_i+'_predictsignal'] = np.where(hit_rate_df[
        ticker_i+'_predictret']>=trading_threshold, 1,hit_rate_df[ticker_i+'_predictsignal'] )
    hit_rate_df[ticker_i+'_predictsignal'] = np.where(hit_rate_df[
        ticker_i+'_predictret']<=(-1)*trading_threshold, -1,hit_rate_df[ticker_i+'_predictsignal'] )
    
     
    hit_rate_df[ticker_i+'_hit'] = np.where(
        np.sign(hit_rate_df[ticker_i+'_actualsignal'])==np.sign(hit_rate_df[ticker_i+'_predictsignal']),
    1, 0)

    hit_rate_df[ticker_i+'_realret'] = hit_rate_df[ticker_i+'_actualret'] * hit_rate_df[ticker_i+'_predictsignal'] 



/tmp/ipykernel_17748/1153982916.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_actualret']  = hit_rate_df[ticker_i+'_actual'].diff().shift(-1)/\
/tmp/ipykernel_17748/1153982916.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_predictret'] = (hit_rate_df[ticker_i+'_predict'].shift(-1) - \
/tmp/ipykernel_17748/1153982916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

/tmp/ipykernel_17748/1153982916.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_predictsignal'] = np.where(hit_rate_df[
/tmp/ipykernel_17748/1153982916.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_hit'] = np.where(
/tmp/ipykernel_17748/1153982916.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

/tmp/ipykernel_17748/1153982916.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_predictret'] = (hit_rate_df[ticker_i+'_predict'].shift(-1) - \
/tmp/ipykernel_17748/1153982916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_actualsignal'] = 0
/tmp/ipykernel_17748/1153982916.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

/tmp/ipykernel_17748/1153982916.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_predictsignal'] = 0
/tmp/ipykernel_17748/1153982916.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_predictsignal'] = np.where(hit_rate_df[
/tmp/ipykernel_17748/1153982916.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

/tmp/ipykernel_17748/1153982916.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_actualsignal'] = np.where(hit_rate_df[
/tmp/ipykernel_17748/1153982916.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hit_rate_df[ticker_i+'_predictsignal'] = 0
/tmp/ipykernel_17748/1153982916.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

...

/tmp/ipykernel_17748/1153982916.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hit_rate_df[ticker_i+'_hit'] = np.where(
/tmp/ipykernel_17748/1153982916.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_hit'] = np.where(
/tmp/ipykernel_17748/1153982916.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fra

/tmp/ipykernel_17748/1153982916.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_actualret']  = hit_rate_df[ticker_i+'_actual'].diff().shift(-1)/\
/tmp/ipykernel_17748/1153982916.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hit_rate_df[ticker_i+'_predictret'] = (hit_rate_df[ticker_i+'_predict'].shift(-1) - \
/tmp/ipykernel_17748/1153982916.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

/tmp/ipykernel_17748/1153982916.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_actualret']  = hit_rate_df[ticker_i+'_actual'].diff().shift(-1)/\
/tmp/ipykernel_17748/1153982916.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hit_rate_df[ticker_i+'_predictret'] = (hit_rate_df[ticker_i+'_predict'].shift(-1) - \
/tmp/ipykernel_17748/1153982916.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

/tmp/ipykernel_17748/1153982916.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hit_rate_df[ticker_i+'_actualsignal'] = 0
/tmp/ipykernel_17748/1153982916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_actualsignal'] = 0
/tmp/ipykernel_17748/1153982916.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

/tmp/ipykernel_17748/1153982916.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_rate_df[ticker_i+'_predictret'] = (hit_rate_df[ticker_i+'_predict'].shift(-1) - \
/tmp/ipykernel_17748/1153982916.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hit_rate_df[ticker_i+'_actualsignal'] = 0
/tmp/ipykernel_17748/1153982916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [102]:
hit_rate_df_1 = hit_rate_df.filter(like='_hit')

In [103]:
hit_rate_df_1 = hit_rate_df_1[hit_rate_df_1.index < '2022-08-01']

In [77]:
exchange_filter = pd.read_csv("exchange_filter.gz")
exchange_filter = exchange_filter[exchange_filter.HEXCD==3]
exchange_filter = exchange_filter[exchange_filter.HTICK.notna()]

In [82]:
exchange_filter.HTICK

,PERMNO,HTICK,HPRIMEXC,HEXCD
26,10026,JJSF,Q,3
32,10032,PLXS,Q,3
44,10044,RMCF,Q,3
66,10066,FKWL,Q,3
107,10107,MSFT,Q,3
...,...,...,...,...
36371,93369,RCM,Q,3
36373,93371,CRMD,Q,3
36398,93397,LMNR,Q,3
36434,93434,SANW,Q,3


In [88]:
nasdaq_stocks = list(pd.DataFrame.from_dict(
    predicted_df[datetime.date(2020, 2, 28)]).columns.intersection(exchange_filter.HTICK))
nasdaq_stocks = ["{}{}".format(i,'_hit') for i in nasdaq_stocks]

In [105]:
hit_rank = (hit_rate_df_1[
    hit_rate_df_1.columns.intersection(nasdaq_stocks)
].sum()/31).sort_values(ascending =False)

In [112]:
realret_df = hit_rate_df.filter(like='_realret')
cumuret_df = realret_df+1

In [127]:
cumuret_df = cumuret_df[cumuret_df.index<'2022-08-01']

nasdaq_stocks = [i[0:-len('_hit')] for i in nasdaq_stocks]    
nasdaq_stocks = ["{}{}".format(i,'_realret') for i in nasdaq_stocks]

In [130]:
final_ret = cumuret_df[
    cumuret_df.columns.intersection(nasdaq_stocks)
].cumprod().iloc[-1,:].sort_values(ascending = False)

In [166]:
print("90% return:", final_ret.quantile(.9))
print("80% return:", final_ret.quantile(.8))
print("70% return:", final_ret.quantile(.7))
print("60% return:", final_ret.quantile(.6))

print("90% 80% 70% 60% hit rate: \n",hit_rank.quantile([.9, .8, .7, .6]))

90% return: 1.3227633627225959
80% return: 1.102047408935772
70% return: 0.9679686189854516
60% return: 0.8539875719847285
90% 80% 70% 60% hit rate: 
 0.9    0.580645
0.8    0.548387
0.7    0.516129
0.6    0.516129
Name: hit_rate, dtype: float64


In [167]:
final_ret.index = final_ret.index.str.removesuffix("_realret")
final_ret.name = 'final_cumuret'

hit_rank.index= hit_rank.index.str.removesuffix("_hit")
hit_rank.name = 'hit_rate'
hit_rank=hit_rank.sort_index()
final_ret=final_ret.sort_index()

In [161]:
portfolio_df = pd.concat([final_ret, hit_rank], axis=1)
portfolio_df = portfolio_df[(portfolio_df.final_cumuret >= final_ret.quantile(.8))
                            &(portfolio_df.hit_rate>= hit_rank.quantile(.8))]
portfolio_df = portfolio_df.sort_values(by = 'final_cumuret', ascending = False)


In [163]:
portfolio_df.head(10)

,final_cumuret,hit_rate
BKNG,2.646674,0.580645
TER,2.542642,0.580645
AVGO,1.734572,0.612903
SBNY,1.547799,0.548387
NTRS,1.530895,0.580645
NFLX,1.528656,0.548387
REGN,1.499196,0.580645
KLAC,1.413220,0.612903
MTCH,1.407544,0.645161
CMCSA,1.396487,0.612903


In [170]:
portfolio_df.to_csv("10stocks.csv")